In [ ]:
%load_ext autoreload
%autoreload 2

: 

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""
from neuron_explainer.activations.activation_records import calculate_max_activation
from neuron_explainer.activations.activations import ActivationRecordSliceParams, mod_load_neuron,load_neuron
# from neuron_explainer.explanations.calibrated_simulator import UncalibratedNeuronSimulator
from neuron_explainer.explanations.explainer import TokenActivationPairExplainer
from neuron_explainer.explanations.prompt_builder import PromptFormat
# from neuron_explainer.explanations.scoring import simulate_and_score
# from neuron_explainer.explanations.simulator import ExplanationNeuronSimulator

EXPLAINER_MODEL_NAME = "gpt-3.5-turbo"
# SIMULATOR_MODEL_NAME = "text-davinci-003"


# test_response = await client.make_request(prompt="test 123<|endofprompt|>", max_tokens=2)
# print("Response:", test_response["choices"][0]["text"])

## Load a neuron record.
filepath = '../data/newdata.json'
neuron_record = mod_load_neuron(filepath)


# Grab the activation records we'll need.
slice_params = ActivationRecordSliceParams(n_examples_per_split=5)
train_activation_records = neuron_record.train_activation_records(
    activation_record_slice_params=slice_params
)
valid_activation_records = neuron_record.valid_activation_records(
    activation_record_slice_params=slice_params
)
# print('done grabbing the activation record')

# Generate an explanation for the neuron.
explainer = TokenActivationPairExplainer(
    model_name=EXPLAINER_MODEL_NAME,
    prompt_format=PromptFormat.HARMONY_V4,
    max_concurrent=1,
)

explanations = await explainer.generate_explanations(
    all_activation_records=train_activation_records,
    max_activation=calculate_max_activation(train_activation_records),
    num_samples=1,
)
assert len(explanations) == 1
explanation = explanations[0]
print(f"{explanation=}")


done with prompt
[{'role': <Role.SYSTEM: 'system'>, 'content': "We're studying neurons in a neural network. Each neuron looks for some particular thing in a short document. Look at the parts of the document the neuron activates for and summarize in a single sentence what the neuron is looking for. Don't list examples of words.\n\nThe activation format is token<tab>activation. Activation values range from 0 to 10. A neuron finding what it's looking for is represented by a non-zero activation value. The higher the activation value, the stronger the match."}, {'role': <Role.USER: 'user'>, 'content': "\n\nNeuron 1\nActivations:\n<start>\nt\t0\nurt\t0\nur\t0\nro\t0\n is\t0\n fab\t0\nulously\t0\n funny\t0\n and\t0\n over\t0\n the\t0\n top\t0\n as\t0\n a\t0\n '\t0\nvery\t0\n sneaky\t0\n'\t1\n but\t0\nler\t0\n who\t0\n excel\t0\ns\t0\n in\t0\n the\t0\n art\t0\n of\t0\n impossible\t0\n disappearing\t6\n/\t0\nre\t0\napp\t0\nearing\t10\n acts\t0\n<end>\n<start>\nesc\t0\naping\t9\n the\t4\n studio

In [28]:
import json
from collections import defaultdict

file = '/Users/benjaminlee/Desktop/automated-interpretability/newdata.json'

with open(file,'r') as file:
    data = json.load(file)
dictionary = defaultdict(list)
most_activated_tokens = {}
for line in data['most_positive_activation_records']:
    for i in range(len(line['tokens'])):
        dictionary[line['tokens'][i]].append(line['activations'][i])
        if line['tokens'][i] in most_activated_tokens:
            most_activated_tokens[line['tokens'][i]] = max(line['activations'][i],most_activated_tokens[line['tokens'][i]])
        else:
            most_activated_tokens[line['tokens'][i]] = line['activations'][i]
results = {}

for word in dictionary:
    if len(dictionary[word]) > 1:
        results[word] = (sum(dictionary[word])/len(dictionary[word]))    
# for word in dictionary:
    # results[word] = (sum(dictionary[word])/len(dictionary[word]))


most_positive = sorted(results, key=results.get, reverse=True)[:5]
most_activated =  sorted(most_activated_tokens,key=most_activated_tokens.get, reverse=True)[:5]
most_frequent = sorted(dictionary,key=lambda key:len(dictionary[key]),reverse=True)[:5]

print('HIGHEST ACTIVATED TOKENS') 
for wd in most_activated:
    print(wd + ' : ' + str(most_activated_tokens[wd]))
print('MOST POSITIVE AVERAGE TOKENS')
print(most_positive)
for wd in most_positive:
    print(wd + ' : ' + str(results[wd]))
print('MOST FREQUENT TOKEN')
for wd in most_frequent:
    print(wd + ' : ' + str(len(dictionary[wd])))


HIGHEST ACTIVATED TOKENS
account : 2.75133276
( : 2.44138598
Message : 2.37940383
found : 2.35543394
message : 2.09572244
MOST POSITIVE AVERAGE TOKENS
['Message', '(', '!', '...', '&&']
Message : 2.297837735
( : 1.9217299085416666
! : 1.77485001
... : 1.52603793
&& : 1.2722299675
MOST FREQUENT TOKEN
( : 48
if : 32
null : 24
!= : 14
) : 13
